In [1]:
import setup

setup.init_django()

In [2]:
from openai import OpenAI
from products.models import Embedding
from decouple import config
EMEDDING_MODEL=config("EMEDDING_MODEL", default="text-embedding-3-small")
RECREATE_DATA=True

In [3]:
client = OpenAI(
    api_key=config("OPENAI_API_KEY")
)
def get_embedding(text, model=EMEDDING_MODEL):
    text = text.replace("\n", " ").strip()
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [6]:
query = "The dog jumped over the green cow"
# query = "The dog jumped over the cat"
query_embedding = get_embedding(query)
# query_embedding

In [9]:
from pgvector.django import CosineDistance
from django.db.models import F

qs = Embedding.objects.annotate(
    distance=CosineDistance('embedding',query_embedding),
    similarity=1 - F("distance")
).order_by("distance")[:2]
for obj in qs:
    print(obj.object_id, obj.content_object.id, obj.content_object.title, obj.distance, obj.similarity * 100)

28 28 Blog Post 4 0.615726500749588 38.4273499250412
1 1 Random not real product 0.615726500749588 38.4273499250412
